<a href="https://colab.research.google.com/github/FariusGitHub/DataScience/blob/master/SOUP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import bs4
import requests
import re
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd
from datetime import date

# GETTING POSSIBLE LIST FROM INDEED MULTIPLE PAGES FOR TORONTO DATA SCIENTIST
DSW1=[]; DST1=[]; DSS1= []; DSL1= []; DSC1=[]; DSD1=[]; DSE1=[]; DATE=[]
soup = bs4.BeautifulSoup(requests.get('https://ca.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=&radius=25&l=toronto%2C+on&fromage=any&limit=50&sort=&psf=advsrch&from=advancedsearch').text, 'lxml')

for l in tqdm(range(0,int(soup.find('div', id='searchCountPages').text.strip('\n Page jobs').split(' of')[1])//50*50,50)):
    start = l
    soup = bs4.BeautifulSoup(requests.get('https://ca.indeed.com/jobs?q=data+\
            scientist&l=Toronto,+ON&start='+str(start)+'&limit=50').text, 'lxml')

 
    jk=[]; fccid=[]; 
    for i in range(len(soup.select('.title'))):
        j = 0 if soup.select('.date')[i].text.strip()[:2].isalpha() \
            else int(soup.select('.date')[i].text.strip()[:2])
        k=(datetime.today().date()-timedelta((j))).strftime("%x")
        jk += [str(soup.find_all("script", text=re.compile("jobmap"), \
              type="text/javascript")[0]).split('\n\n')[i+3]\
              .split("]= {jk:'")[1].split(",cmpid:'")[0].split("',efccid:")[0]]
        fccid += [str(soup.find_all("script", text=re.compile("jobmap"), \
                type="text/javascript")[0]).split('\n\n')[i+3]\
                .split("]= {jk:'")[1].split(",cmpid:'")[1].split("',num:'")[0]]
        DSW1+=['http://ca.indeed.com/rc/clk?jk='+jk[i]+'&fccid='+fccid[i]+'&vjs=3'] #WEB
        DST1+=[soup.select('.title')[i].a.get('title')]                             #TITLE
        DSS1+=[soup.select('.summary')[i].text.strip('\n')]                         #SUMMARY
        DSL1+=[soup.select('.sjcl')[i](class_='recJobLoc')[0].get('data-rc-loc')]   #LOC
        DSC1+=[soup.select('.sjcl')[i](class_='company')[0].text.strip('\n')]       #COMPANY 
        DSD1+=['>=30days' if j == 30 else k]                                        #DURATION 
        DATE+=[date.today()]

# GETTING JOB DESC FROM INDIVIDUAL JOB SITE ABOVE
DSE1=[]
print(len(DSW1))
for i in tqdm(range(len(DSW1))):
    try:
        DSE1 += ['<p>'.join(str(bs4.BeautifulSoup(requests.get(DSW1[i])\
        .text, 'lxml').select(".jobsearch-jobDescriptionText")[0])\
        .split('</p>')).lstrip('<div class="jobsearch-jobDescriptionText" id=\
              "jobDescriptionText"><p>')]
    except Exception:
        DSE1 += ['error during downloading']            #DESC

# PREPARING REPORT AND OPEN EXCEL        
df = pd.DataFrame(list(zip(DST1, DSC1, DSL1, DSS1, DSE1, DATE, DSD1, DSW1)),
      columns =['Title', 'Company', 'Location', 'Summary', 'Description', 'Date', 'Duration', 'Link'])

df.style.hide_index()